In [1]:
import pandas
import os
import sys
import requests
import time
import pandas

# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials

from IPython.display import display, clear_output

## Downloading Failed CSV files

In [8]:
regional_error = os.listdir("data/regional/logs")
error_types = {}
for file in regional_error:
    file_type = file.partition("_")[0]
    if file_type in error_types:
        error_types[file_type] += 1
    else:
        error_types[file_type] = 1
# regional_access_error = [x for x in regional_error if "access-error" in x]
# len(regional_access_error)
print(error_types)

{'access-error': 100, 'missing': 9747, 'not-exist': 199, 'unexpected-html': 15, 'unexpected-start': 1}


In [21]:
output_dir = "data/regional/attempt2"
logdir2 = "data/regional/attempt2/logs"
for folder in (output_dir, logdir2):
    if not (os.path.isdir(folder)):
        os.mkdir(folder)

REGIONAL_HEADER = 'Position,"Track Name",Artist,Streams,URL'
VIRAL_HEADER = 'Position,"Track Name",Artist,URL'
HTML_HEADER = '<!doctype html>'
PREFACE_LINE = ',,,"Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.",'
        
file_data_list = []
error_types = {}
for file in os.listdir("data/regional/logs"):
    file_type, _, file_data = file.partition("_")
    if file_type in error_types:
        error_types[file_type] += 1
    else:
        error_types[file_type] = 1
    if file_type in ('access-error', 'unexpected-html', 'unexpected-start'):
        file_data_list.append(  (file_type, file_data) )
    
total_files = error_types['access-error'] + error_types['unexpected-html'] + error_types['unexpected-start']

count = 0

for err_type, file_data in file_data_list:
    count += 1
    
    country_and_date = file_data.partition(".txt")[0]
    country, _, date = country_and_date.partition("_")
    
    attempt = 0
    
    request_url = "https://spotifycharts.com/regional/{}/daily/{}/download".format(country, date)
    logfile = os.path.join(logdir2, "{}_{}_{}.txt".format(err_type, country, date))
    filepath = os.path.join(output_dir, "regional_{}_{}.csv".format(country, date))
    
    success = False
    
    while attempt <= 20 and not success:
        attempt += 1
        
        sys.stdout.write("\r{} / {}, attempt: {}".format(count, total_files, attempt))
        sys.stdout.flush()
        
        r = requests.get(request_url)
        
        if r:
            raw_text = r.text
            first_line, _, remaining_text = raw_text.partition("\n")
            if first_line in (REGIONAL_HEADER, VIRAL_HEADER, PREFACE_LINE):
                
                # File has successfully been downloaded. Need to save it to disk.
                if first_line in (REGIONAL_HEADER, VIRAL_HEADER):
                    output_text = raw_text
                else:
                    output_text = remaining_text
                
                with open(filepath, "wb") as f:
                    f.write(output_text.encode('utf-8'))
                
                success = True
                break
        time.sleep(1)
    
    if not success:
        # The download has failed
        with open(logfile, "wb") as f:
            f.write("0".encode('utf-8'))
            

116 / 116, attempt: 1

## Copying re-attempted files across

In [22]:
success_files = os.listdir("data/regional/attempt2")
for file in success_files:
    if "regional_" in file:
        filepath = os.path.join("data/regional/attempt2", file)
        country = file.split("_")[1]
        destination = os.path.join("data/regional", country, file)
        os.replace(filepath, destination)
        

## Creating master files

In [29]:
subfolders = [f.name for f in os.scandir("data/regional") if (f.is_dir() and not f.name in ('attempt2', 'logs'))]
subfolders

for country in subfolders:
    df = pd.DataFrame()
    folder_path = os.path.join("data/regional", country)
    file_names = os.listdir(folder_path)
    for file in file_names:
        fdata = file.partition(".csv")
        date = fdata.split("_")[-1]
        temp_df = pandas.read_csv(os.path.join(folder_path, file))
        

['ad',
 'ar',
 'at',
 'au',
 'be',
 'bg',
 'bo',
 'br',
 'ca',
 'ch',
 'cl',
 'co',
 'cr',
 'cy',
 'cz',
 'de',
 'dk',
 'do',
 'ec',
 'ee',
 'es',
 'fi',
 'fr',
 'gb',
 'global',
 'gr',
 'gt',
 'hk',
 'hn',
 'hu',
 'id',
 'ie',
 'il',
 'in',
 'is',
 'it',
 'jp',
 'lt',
 'lu',
 'lv',
 'mc',
 'mt',
 'mx',
 'my',
 'ni',
 'nl',
 'no',
 'nz',
 'pa',
 'pe',
 'ph',
 'pl',
 'pt',
 'py',
 'ro',
 'se',
 'sg',
 'sk',
 'sv',
 'th',
 'tr',
 'tw',
 'us',
 'uy',
 'vn',
 'za']

In [36]:
df = pandas.read_csv("data/regional/global/regional_global_2017-01-01.csv")
df.head()

,Position,Track Name,Artist,Streams,URL
0,1,Starboy,The Weeknd,3135625,https://open.spotify.com/track/5aAx2yezTd8zXrk...
1,2,Closer,The Chainsmokers,3015525,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...
2,3,Let Me Love You,DJ Snake,2545384,https://open.spotify.com/track/4pdPtRcBmOSQDlJ...
3,4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,https://open.spotify.com/track/5knuzwU65gJK7IF...
4,5,One Dance,Drake,2259887,https://open.spotify.com/track/1xznGGDReH1oQq0...


In [39]:
chart_data = {}
for index, row in df.iterrows():
    position = row['Position']
    track_name = row['Track Name']
    artist_name = row['Artist']
    streams = row['Streams']
    url = row['URL']
    song_id = url.split("/")[-1]
    chart_data[position] = {
        "track_name": track_name,
        "artist_name": artist_name,
        "streams": streams,
        "song_id": song_id
    }

print(chart_data)
df2 = pandas.DataFrame(chart_data)

{1: {'track_name': 'Starboy', 'artist_name': 'The Weeknd', 'streams': 3135625, 'song_id': '5aAx2yezTd8zXrkmtKl66Z'}, 2: {'track_name': 'Closer', 'artist_name': 'The Chainsmokers', 'streams': 3015525, 'song_id': '7BKLCZ1jbUBVqRi2FVlTVw'}, 3: {'track_name': 'Let Me Love You', 'artist_name': 'DJ Snake', 'streams': 2545384, 'song_id': '4pdPtRcBmOSQDlJ3Fk945m'}, 4: {'track_name': 'Rockabye (feat. Sean Paul & Anne-Marie)', 'artist_name': 'Clean Bandit', 'streams': 2356604, 'song_id': '5knuzwU65gJK7IF5yJsuaW'}, 5: {'track_name': 'One Dance', 'artist_name': 'Drake', 'streams': 2259887, 'song_id': '1xznGGDReH1oQq0xzbwXa3'}, 6: {'track_name': 'Fake Love', 'artist_name': 'Drake', 'streams': 2137437, 'song_id': '343YBumqHu19cGoGARUTsd'}, 7: {'track_name': '24K Magic', 'artist_name': 'Bruno Mars', 'streams': 2111599, 'song_id': '6b8Be6ljOzmkOmFslEb23P'}, 8: {'track_name': "Don't Wanna Know", 'artist_name': 'Maroon 5', 'streams': 2107137, 'song_id': '5MFzQMkrl1FOOng9tq6R9r'}, 9: {'track_name': 'Blac

## Downloading Song and Artist information

In [46]:
track_set = set()
artist_set = set()

subfolders = [f.name for f in os.scandir("data/regional") if (f.is_dir() and not f.name in ('attempt2', 'logs'))]
subfolders

for folder in subfolders:
    country_folder = os.path.join("data/regional", folder)
    file_list = os.listdir(country_folder)
    for file in file_list:
        df_temp = pandas.read_csv(os.path.join(country_folder, file))
        df_songs = df_temp.loc[ df_temp['URL'].notnull() ]
        for url in df_songs['URL'].tolist():
            track_id = url.split("/")[-1]
            track_set.add(track_id)
                
print(len(track_set))


69688


In [135]:
track_data_file = "data/regional/track_ids"
with open(track_data_file, "w") as f:
    for track_id in track_set:
        text = str(track_id) + "\n"
        f.write(text)

In [50]:
test_id = next(iter(track_set))

In [52]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="37bf09c4a866456ebea69781382fb291" 
secret = "04a9cd8f2f0e42c3b7072d58932d03a5"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [134]:
tracks = iter(track_set)
song_ids = [next(tracks) for i in range(50)]
track_list = sp.tracks(song_ids)
feature_list = sp.audio_features(song_ids)
feature_list

[{'danceability': 0.479,
  'energy': 0.654,
  'key': 7,
  'loudness': -6.858,
  'mode': 1,
  'speechiness': 0.382,
  'acousticness': 0.56,
  'instrumentalness': 0,
  'liveness': 0.084,
  'valence': 0.754,
  'tempo': 81.952,
  'type': 'audio_features',
  'id': '1Y2CGJS28CtJHMVj06vZx8',
  'uri': 'spotify:track:1Y2CGJS28CtJHMVj06vZx8',
  'track_href': 'https://api.spotify.com/v1/tracks/1Y2CGJS28CtJHMVj06vZx8',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1Y2CGJS28CtJHMVj06vZx8',
  'duration_ms': 200733,
  'time_signature': 4},
 {'danceability': 0.704,
  'energy': 0.871,
  'key': 2,
  'loudness': -6.114,
  'mode': 1,
  'speechiness': 0.0514,
  'acousticness': 0.159,
  'instrumentalness': 0,
  'liveness': 0.154,
  'valence': 0.95,
  'tempo': 132.006,
  'type': 'audio_features',
  'id': '3KQ5zUYkmrvF3gQojLm2zN',
  'uri': 'spotify:track:3KQ5zUYkmrvF3gQojLm2zN',
  'track_href': 'https://api.spotify.com/v1/tracks/3KQ5zUYkmrvF3gQojLm2zN',
  'analysis_url': 'https://api.spotify.co

In [127]:
df = pandas.DataFrame()
count = 0
for track_id in track_set:
    count += 1
    if count <= 3400:
        continue
        
    if count % 100 == 0:
        clear_output(True)
        display(count)
        
    
    track_info = sp.track(track_id)
    track_features = sp.audio_features(track_id)[0]
    
    track_dict = {
        "id": track_id,
        "name": track_info['name'],
        "explicit": track_info['explicit'],
        "duration": track_info['duration_ms'],
        "current_popularity": track_info['popularity'],
        "available_markets": tuple(track_info['available_markets']),
        "artist_id": track_info['artists'][0]['id'],
        "artist_name": track_info['artists'][0]['name'],
        "album_id": track_info['album']['id'],
        "album_name": track_info['album']['name'],
        "album_tracks": track_info['album']['total_tracks'],
        "track_number": track_info['track_number'],
        "album_release_date": track_info['album']['release_date'],
        "release_date_precision": track_info['album']['release_date_precision']
    }
    
    if len(track_info['artists']) > 1:
        track_dict['additional_artists'] = tuple([(artist['name'], artist['id']) for artist in track_info['artists'][1:]])
    else:
        track_dict['additional_artists'] = None
    
    if track_features != None:
        for data_label in ('danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness',
                            'liveness','valence','tempo','time_signature'):
            track_dict[data_label] = track_features[data_label]
    
    for artist in track_info['artists']:
        artist_id = artist['id']
        artist_set.add(artist_id)
    
    temp_df = pandas.DataFrame([track_dict])
    df = pandas.concat([df, temp_df])



3600

retrying ...5secs
retrying ...5secs


KeyboardInterrupt: 

In [ ]:
with open("track_info.csv", "w") as f:
    f.write(df.to_csv(index=False))

In [101]:
track_info = sp.track(test_id)
track_features = sp.audio_features(test_id)

In [102]:
track_features

[{'danceability': 0.479,
  'energy': 0.654,
  'key': 7,
  'loudness': -6.858,
  'mode': 1,
  'speechiness': 0.382,
  'acousticness': 0.56,
  'instrumentalness': 0,
  'liveness': 0.084,
  'valence': 0.754,
  'tempo': 81.952,
  'type': 'audio_features',
  'id': '1Y2CGJS28CtJHMVj06vZx8',
  'uri': 'spotify:track:1Y2CGJS28CtJHMVj06vZx8',
  'track_href': 'https://api.spotify.com/v1/tracks/1Y2CGJS28CtJHMVj06vZx8',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1Y2CGJS28CtJHMVj06vZx8',
  'duration_ms': 200733,
  'time_signature': 4}]

In [60]:
track_info.keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [84]:
track_info['id']

'1Y2CGJS28CtJHMVj06vZx8'

In [68]:
track_info['album'].keys()

dict_keys(['album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])

In [82]:
track_info['album']['total_tracks']

15

In [89]:
artist_id = track_info['artists'][0]['id']
artist_info = sp.artist(artist_id)

In [90]:
artist_info.keys()

dict_keys(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name', 'popularity', 'type', 'uri'])

In [100]:
artist_info['genres']

['french hip hop', 'pop urbaine']

---

In [156]:
subfolders = [f.name for f in os.scandir("data/regional") if (f.is_dir() and not f.name in ('attempt2', 'logs'))]
subfolders

for country in subfolders:
    country_folder = os.path.join("data/regional", country)
    file_list = os.listdir(country_folder)
    
    if len(file_list) == 0:
        continue
    
    chart_data_list = []
    
    for file in file_list:
        date = file.split("_")[-1][:-4]
        df_temp = pandas.read_csv(os.path.join(country_folder, file))
        for index, row in df_temp.iterrows():
            position = row['Position']
            track_name = row['Track Name']
            artist_name = row['Artist']
            streams = row['Streams']
            track_url = row['URL']
            if isinstance(track_url, str):
                track_id = track_url.split("/")[-1]
            else:
                track_id = None
                
            chart_data = {
                "date": date,
                "position": position,
                "track_name": track_name,
                "artist_name": artist_name,
                "streams": streams,
                "track_id": track_id,
                "country": country
            }
            chart_data_list.append(chart_data)
            
        df = pandas.DataFrame(chart_data_list)
    
    csv_path = os.path.join("data", "regional", country + ".csv")
    
    df.to_csv(csv_path, index=False)
    
df.head()
        
        


KeyboardInterrupt: 

In [ ]:
output_csv = os.path.join("data", "regional", "all_charts.csv")
df.to_csv(output_csv, index=False)

In [152]:
df.loc[ df['position'] == 199 ]

,date,position,track_name,artist_name,streams,track_id,country


In [ ]:
#         df_songs = df_temp.loc[ df_temp['URL'].notnull() ]
#         for url in df_songs['URL'].tolist():
#             track_id = url.split("/")[-1]
#             track_set.add(track_id)



# chart_data = {}
# for index, row in df.iterrows():
#     pos


# chart_data = {}
# for index, row in df.iterrows():
#     position = row['Position']
#     track_name = row['Track Name']
#     artist_name = row['Artist']
#     streams = row['Streams']
#     url = row['URL']
#     song_id = url.split("/")[-1]
#     chart_data[position] = {
#         "track_name": track_name,
#         "artist_name": artist_name,
#         "streams": streams,
#         "song_id": song_id
#     }

# print(chart_data)
# df2 = pandas.DataFrame(chart_data)

In [144]:
for index, row in df_temp.iterrows():
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


 # TESTING

In [29]:
artist_set = set()
track_set = set()
file_list = os.listdir("data/regional/global")
len(file_list)

1152

In [35]:
for file in file_list:
    sys.stdout.write("\r"+file)
    sys.stdout.flush()
    filepath = "data/regional/global/" + file
    try:
        df = pandas.read_csv(filepath, comment="#")
        for track_url in df['URL']:
            track_id = track_url.split('/')[-1]
            track_set.add(track_id)
    except Exception as e:
        print(e)

regional_global_2018-01-30.csvError tokenizing data. C error: Expected 1 fields in line 6, saw 2

regional_global_2019-08-23.csv'float' object has no attribute 'split'
regional_global_2020-03-01.csv

In [36]:
len(track_set)

4837

In [63]:
from bs4 import BeautifulSoup as bs
with open("data/logs/regional_cy_2017-08-23.txt", "r") as f:
    xdata = f.read()
soup = bs(xdata)
b = soup.find("div", attrs={"class": "chart-error"})
b.text.strip()

'This chart does not exist. Please make another selection from the dropdown menus.'

In [65]:
c = None.text.strip()

AttributeError: 'NoneType' object has no attribute 'text'

In [28]:
import requests
r = requests.get("https://spotifycharts.com/viral/id/daily/2017-01-01")
r_soup = bs(r.text)
print(r_soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   Spotify Charts
  </title>
  <meta content="Spotify Charts" property="og:title"/>
  <meta content="website" property="og:type"/>
  <meta content="https://www.spotifycharts.com/" property="og:url"/>
  <meta content="http://d2c87l0yth4zbw-2.global.ssl.fastly.net/i/_global/open-graph-default.png" property="og:image"/>
  <meta content="Spotify is all the music you’ll ever need." property="og:description"/>
  <meta content="Spotify is all the music you’ll ever need." name="description"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="/images/touch-icon-144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
  <link href="/images/touch-icon-114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
  <link href="/images/touch-icon-72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
  <link href="/images

In [58]:
a = r_soup.find_all("div", id='content')
c = a[0].find_parents("div")
print(len(c))
print(a[0].prettify())
q = r_soup.find("div", attrs={"class":"chart-error"})
q == None

0
<div id="content">
 <div>
  <header class="header">
   <div class="container">
    <a class="header-csv" download="" href="/viral/id/daily/2017-01-01/download">
     Download to CSV
    </a>
    <h1 class="header-logo">
     <a href="/">
      Spotify
     </a>
    </h1>
    <h2 class="header-title">
     Charts
    </h2>
   </div>
  </header>
  <div class="wrapper">
   <div class="container">
    <span>
     <div class="wrapper">
      <div class="container">
       <div class="chart-filters">
        <label>
         Filter by
        </label>
        <a class="menu-button">
         Menu
        </a>
        <div class="chart-filters-list">
         <div class="responsive-select" data-type="country">
          <div class="responsive-select-value">
           Indonesia
          </div>
          <ul>
           <li class="" data-value="global">
            Global
           </li>
           <li class="" data-value="us">
            United States
           </li>
           <li clas

True

In [56]:
import re
t = r_soup.find_all(attrs={"class":re.compile("chart-*")})
t

[<div class="chart-filters">
 <label>Filter by</label>
 <a class="menu-button">Menu</a>
 <div class="chart-filters-list">
 <div class="responsive-select" data-type="country">
 <div class="responsive-select-value">Indonesia</div>
 <ul>
 <li class="" data-value="global">Global</li>
 <li class="" data-value="us">United States</li>
 <li class="" data-value="gb">United Kingdom</li>
 <li class="" data-value="ad">Andorra</li>
 <li class="" data-value="ar">Argentina</li>
 <li class="" data-value="at">Austria</li>
 <li class="" data-value="au">Australia</li>
 <li class="" data-value="be">Belgium</li>
 <li class="" data-value="bg">Bulgaria</li>
 <li class="" data-value="bo">Bolivia</li>
 <li class="" data-value="br">Brazil</li>
 <li class="" data-value="ca">Canada</li>
 <li class="" data-value="ch">Switzerland</li>
 <li class="" data-value="cl">Chile</li>
 <li class="" data-value="co">Colombia</li>
 <li class="" data-value="cr">Costa Rica</li>
 <li class="" data-value="cy">Cyprus</li>
 <li class

In [7]:
df = pandas.read_csv("regional_do_2017-01-01.csv")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 7, saw 2


In [6]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,Chantaje (feat. Maluma),Shakira,7707,https://open.spotify.com/track/6mICuAdrwEjh6Y6...
2,2,Otra vez (feat. J Balvin),Zion & Lennox,7267,https://open.spotify.com/track/3QwBODjSEzelZyV...
3,3,Dile Que Tu Me Quieres,Ozuna,7005,https://open.spotify.com/track/20ZAJdsKB5IGbGj...
4,4,"Diles (feat. Arcangel, Nengo Flow, Dj Luian & ...",Ozuna,6606,https://open.spotify.com/track/2FrnTVSHjgnEylK...


In [8]:
track_id = df['URL'][0]

In [9]:
track_id

'https://open.spotify.com/track/5aAx2yezTd8zXrkmtKl66Z'

In [10]:
track_url = track_id

In [11]:
track_url

'https://open.spotify.com/track/5aAx2yezTd8zXrkmtKl66Z'

In [12]:
track_id = track_url.split("/")[-1]

In [13]:
track_id

'5aAx2yezTd8zXrkmtKl66Z'

In [18]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="37bf09c4a866456ebea69781382fb291" 
secret = "04a9cd8f2f0e42c3b7072d58932d03a5"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [27]:
sp.artist('1Xyo4u8uXC1ZmMpatF05PJ')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
 'followers': {'href': None, 'total': 19046298},
 'genres': ['canadian contemporary r&b', 'canadian pop', 'pop'],
 'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
 'id': '1Xyo4u8uXC1ZmMpatF05PJ',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/d9a875c37277c35b94c60c00d2cd256db098505d',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/91c55730d3f156894d744d560afe441af3f5a228',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/19393c56208fe6c312384879c11ff3da463f1304',
   'width': 160}],
 'name': 'The Weeknd',
 'popularity': 96,
 'type': 'artist',
 'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}